<a href="https://colab.research.google.com/github/vriadi/CS614-Gen-AI-with-LLMs/blob/main/Topic8_RAG_Simple_empty.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Create your own prompt and your own "mini-documents" by replacing the below phrases with your own.  The key is to (1) think of a prompt, then (2) think of a few possible "documents" (answers) to your prompt, some of which are obviously false, some which are ok, but not a great answer, and then at least 1 which is a very good answer.

We will use RAG to see if we can obtain the best answer.

In [1]:
# prompt = "Where can I learn Gen AI in Singapore?"

In [2]:
# documents = ["My cat's name is Spot. What is your name?",
#              "SMU is a university in Singapore with degrees in management and information systems and computer science",
#              "I am taking a fabulous course on Gen AI at SMU and learning so many useful things!",
#              "Je connais une universite specialisee en commerce et informatique a Singapour qui s'appelle SMU ou il y a un excellent cours en intelligence artificielle.",
#              "I would like to learn Gen AI as I believe it will help me in my career in the near future."]

In [1]:
prompt = "How do I edit the test drive consent?"

In [2]:
documents = ["My dog's name is Drive. He really like to test my patience. What is your name?",
             "I update consents by going to the consent setting. There I can change or update the translation.",
             "Consent is a very important thing in every industry.",
             "Für die Probefahrt mit einem Auto ist in der Regel eine Probefahrtgenehmigung erforderlich.",
             "I would like to learn how to draft up a consent especially in the legal industry as I believe it will help me in my career in the near future.",
             "I tested my car to drive up to 180 on the race track without my parents consent."]

In [3]:
!pip install sentence_transformers
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
prompt_embed = model.encode(prompt)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Model comes from https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2

In [4]:
doc_embed = model.encode(documents)
len(doc_embed[0])

384

In [5]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [11]:
sim = cosine_similarity(np.array([prompt_embed]),doc_embed)
top_index = np.argmax(sim)
top_doc = documents[top_index]
print(f"Prompt: {prompt}\n Top response: {top_doc}")

Prompt: How do I edit the test drive consent?
 Top response: I would like to learn how to draft up a consent especially in the legal industry as I believe it will help me in my career in the near future.


In [12]:
sorted = np.argsort(sim[0])[::-1]

ranking = [(documents[i], sim[0][i]) for i in sorted]
ranking

[('I would like to learn how to draft up a consent especially in the legal industry as I believe it will help me in my career in the near future.',
  np.float32(0.51729494)),
 ('I update consents by going to the consent setting. There I can change or update the translation.',
  np.float32(0.46745652)),
 ('I tested my car to drive up to 180 on the race track without my parents consent.',
  np.float32(0.42003354)),
 ('Consent is a very important thing in every industry.',
  np.float32(0.34900218)),
 ("My dog's name is Drive. He really like to test my patience. What is your name?",
  np.float32(0.23873807)),
 ('Für die Probefahrt mit einem Auto ist in der Regel eine Probefahrtgenehmigung erforderlich.',
  np.float32(0.03535021))]

NOW LET'S LOOK AT USING CROSS_ENCODER TO IMPROVE RAG VIA RE-RANKING
https://www.sbert.net/examples/applications/cross-encoder/README.html


In [25]:
from sentence_transformers import CrossEncoder
#cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')
cross_encoder = CrossEncoder('cross-encoder/mmarco-mMiniLMv2-L12-H384-v1')

In [27]:
pairs = []
for doc in documents:
  pairs.append([prompt, doc])

In [28]:
scores = cross_encoder.predict(pairs)

print(f"Prompt: {prompt}\n")
for doc, score in zip(documents, scores):
  print(f"Document: {doc}\nScore: {score}\n")

Prompt: How do I edit the test drive consent?

Document: My dog's name is Drive. He really like to test my patience. What is your name?
Score: -10.244443893432617

Document: I update consents by going to the consent setting. There I can change or update the translation.
Score: -2.803077220916748

Document: Consent is a very important thing in every industry.
Score: -6.966379642486572

Document: Für die Probefahrt mit einem Auto ist in der Regel eine Probefahrtgenehmigung erforderlich.
Score: -3.4625396728515625

Document: I would like to learn how to draft up a consent especially in the legal industry as I believe it will help me in my career in the near future.
Score: -7.0620808601379395

Document: I tested my car to drive up to 180 on the race track without my parents consent.
Score: -7.028048992156982



In [29]:
top_index_cross = np.argmax(scores)
top_doc_cross = documents[top_index_cross]
print(f"Prompt: {prompt}\n Top response: {top_doc_cross}")

Prompt: How do I edit the test drive consent?
 Top response: I update consents by going to the consent setting. There I can change or update the translation.


In [8]:
!pip install cohere

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.4/295.4 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 73.5 MB/s eta 0:00:00
  Attempting uninstall: httpx-sse
    Found existing installation: httpx-sse 0.4.1
    Uninstalling httpx-sse-0.4.1:
      Successfully uninstalled httpx-sse-0.4.1


In [16]:
import cohere
co = cohere.Client("bssZhmqFD0PxuPi6iyPvwikVqf5IuXlynXFs3Vj0")

response = co.rerank(
    model="rerank-english-v3.0",
    query=prompt,
    documents=documents,
    return_documents=True
)

response

RerankResponse(id='95e1f2c7-3eaf-45c9-8e92-3f4b87cbc1f8', results=[RerankResponseResultsItem(document=RerankResponseResultsItemDocument(text='I update consents by going to the consent setting. There I can change or update the translation.'), index=1, relevance_score=0.24944726), RerankResponseResultsItem(document=RerankResponseResultsItemDocument(text='Für die Probefahrt mit einem Auto ist in der Regel eine Probefahrtgenehmigung erforderlich.'), index=3, relevance_score=0.0006615174), RerankResponseResultsItem(document=RerankResponseResultsItemDocument(text='Consent is a very important thing in every industry.'), index=2, relevance_score=0.00028685408), RerankResponseResultsItem(document=RerankResponseResultsItemDocument(text='I tested my car to drive up to 180 on the race track without my parents consent.'), index=5, relevance_score=0.00013873412), RerankResponseResultsItem(document=RerankResponseResultsItemDocument(text='I would like to learn how to draft up a consent especially in t